In [1]:
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.8/544.8 kB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.7/131.7 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 14.3 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0


In [2]:
from unsloth import FastLanguageModel
from datasets import load_dataset ,Dataset,concatenate_datasets
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "google/gemma-2b-it",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.8.10: Fast Gemma patching. Transformers: 4.55.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.07G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16, lora_alpha=32, lora_dropout=0.05,
    target_modules=["q_proj", "v_proj"],
)
# Due to low VRAM

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.8.10 patched 18 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [5]:
from peft import PeftModel
print(isinstance(model, PeftModel))

True


# load dataset

### Tatoeba Dataset

In [6]:
dataset = load_dataset("opus100", "en-fa")
print(dataset)

README.md: 0.00B [00:00, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/193k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/82.4M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/190k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['translation'],
        num_rows: 2000
    })
    train: Dataset({
        features: ['translation'],
        num_rows: 1000000
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 2000
    })
})


In [7]:
type(dataset)

datasets.dataset_dict.DatasetDict

In [8]:
dataset["train"]

Dataset({
    features: ['translation'],
    num_rows: 1000000
})

In [9]:
dataset["train"]["translation"]

[{'en': 'Pack your stuff.', 'fa': 'وسايلتو جمع کن باشه.'},
 {'en': 'I remember the cow that stayed over there.',
  'fa': 'يادمه يه گاو اونجا بود'},
 {'en': '_', 'fa': 'و رييس سازمان "ياماگاتو" هم بين کشته ها هستن'},
 {'en': 'Oh', 'fa': 'اِی #@%$* توش، غارِ یخـی؟'},
 {'en': 'Aunt Silv, stop yelling!', 'fa': 'عمه سيلو داد نزن'},
 {'en': 'I need to get out of here.', 'fa': 'بايد از اينجا بيام بيرون.'},
 {'en': 'Which means the mommy of the smartest physicist at the university is not my mommy as I had thought.',
  'fa': 'که يعني اونطور که فکر ميکردم مامان باهوش ترين فيزيکدان توي دانشگاه مامان من نيست.'},
 {'en': "That's him.", 'fa': 'خودشه'},
 {'en': '- Hey.', 'fa': '-سلام'},
 {'en': "If it wasn't for Harry we wouldn't even know Voldemort was back.",
  'fa': 'اگه هري نبود کسي نميفهميد که ولدمورت برگشته!'},
 {'en': "It's what I want you to do.",
  'fa': 'اين چيزيه که من ازت مي خوام انجام بدي.'},
 {'en': "You've been the principality priest for a long time.",
  'fa': 'تو خيلي وقته که کشيش اي

In [10]:
dataset["train"].column_names


['translation']

In [11]:
def format_tatoeba(example):
    return {
        "input_text": f"Translate English to Persian: {example['translation']['en']}",
        "output_text": example['translation']['fa'],
    }

tatoeba = dataset["train"].map(format_tatoeba)

Map:   0%|          | 0/1000000 [00:00<?, ? examples/s]

In [12]:
tatoeba["input_text"]

['Translate English to Persian: Pack your stuff.',
 'Translate English to Persian: I remember the cow that stayed over there.',
 'Translate English to Persian: _',
 'Translate English to Persian: Oh',
 'Translate English to Persian: Aunt Silv, stop yelling!',
 'Translate English to Persian: I need to get out of here.',
 'Translate English to Persian: Which means the mommy of the smartest physicist at the university is not my mommy as I had thought.',
 "Translate English to Persian: That's him.",
 'Translate English to Persian: - Hey.',
 "Translate English to Persian: If it wasn't for Harry we wouldn't even know Voldemort was back.",
 "Translate English to Persian: It's what I want you to do.",
 "Translate English to Persian: You've been the principality priest for a long time.",
 'Translate English to Persian: Again?',
 'Translate English to Persian: Anna!',
 'Translate English to Persian: Ha!',
 "Translate English to Persian: It's cold out here.",
 'Translate English to Persian: What 

In [13]:
tatoeba["output_text"]

['وسايلتو جمع کن باشه.',
 'يادمه يه گاو اونجا بود',
 'و رييس سازمان "ياماگاتو" هم بين کشته ها هستن',
 'اِی #@%$* توش، غارِ یخـی؟',
 'عمه سيلو داد نزن',
 'بايد از اينجا بيام بيرون.',
 'که يعني اونطور که فکر ميکردم مامان باهوش ترين فيزيکدان توي دانشگاه مامان من نيست.',
 'خودشه',
 '-سلام',
 'اگه هري نبود کسي نميفهميد که ولدمورت برگشته!',
 'اين چيزيه که من ازت مي خوام انجام بدي.',
 'تو خيلي وقته که کشيش اين منطقه هستي',
 'دوباره؟',
 'آنا!',
 'ها!',
 '. تو منو نمیخوای',
 'چرا اين درس ؟',
 'متأسفم.',
 'محتویات',
 'چرا باید به مردی اعتماد کنم که به سنت احترام نمیذاره؟',
 'از سرزنش کردن خودت دست بردار.',
 'اون جادوگر کجا رفت؟',
 'چه کمکی میتونم کنم؟',
 'وحشتناکه.',
 'صبح بخير. ما در خونه آقاي "لئوپالدو پيزانلو" هستيم',
 'البته اگه خيلي دير نشده باشه.',
 'متأسفم.',
 'اه،خداي من',
 'آخ!',
 'سلبي ؟',
 'من نمي خوامش',
 'چی تورو خوشحال میکنه؟',
 'تو پشت همه ی این قضایایی',
 'و الان، هر دوشون ارتش هاي بزرگتر دارن',
 'و آنچه در آسمان ها و زمین از جنبندگان و فرشتگان وجود دارد، فقط برای خدا سجده می کنن

In [14]:
tatoeba["translation"]


[{'en': 'Pack your stuff.', 'fa': 'وسايلتو جمع کن باشه.'},
 {'en': 'I remember the cow that stayed over there.',
  'fa': 'يادمه يه گاو اونجا بود'},
 {'en': '_', 'fa': 'و رييس سازمان "ياماگاتو" هم بين کشته ها هستن'},
 {'en': 'Oh', 'fa': 'اِی #@%$* توش، غارِ یخـی؟'},
 {'en': 'Aunt Silv, stop yelling!', 'fa': 'عمه سيلو داد نزن'},
 {'en': 'I need to get out of here.', 'fa': 'بايد از اينجا بيام بيرون.'},
 {'en': 'Which means the mommy of the smartest physicist at the university is not my mommy as I had thought.',
  'fa': 'که يعني اونطور که فکر ميکردم مامان باهوش ترين فيزيکدان توي دانشگاه مامان من نيست.'},
 {'en': "That's him.", 'fa': 'خودشه'},
 {'en': '- Hey.', 'fa': '-سلام'},
 {'en': "If it wasn't for Harry we wouldn't even know Voldemort was back.",
  'fa': 'اگه هري نبود کسي نميفهميد که ولدمورت برگشته!'},
 {'en': "It's what I want you to do.",
  'fa': 'اين چيزيه که من ازت مي خوام انجام بدي.'},
 {'en': "You've been the principality priest for a long time.",
  'fa': 'تو خيلي وقته که کشيش اي

### JFLEG Dataset

In [15]:
src_file = "/content/drive/MyDrive/Mathematical_Foundations_and_Artificial_Intelligence_of_Source_Codes/fine-tune lora/Fine tune gemma/dataset/jfleg-master/jfleg-master/test/test.src"
ref_file = "/content/drive/MyDrive/Mathematical_Foundations_and_Artificial_Intelligence_of_Source_Codes/fine-tune lora/Fine tune gemma/dataset/jfleg-master/jfleg-master/test/test.ref0"

with open(src_file, "r", encoding="utf-8") as f:
    src_sentences = f.readlines()

with open(ref_file, "r", encoding="utf-8") as f:
    ref_sentences = f.readlines()

grammar_data = []
for src, ref in zip(src_sentences, ref_sentences):
    grammar_data.append({
        "input_text": f"Correct this sentence: {src.strip()}",
        "output_text": ref.strip()
    })


In [16]:
grammar_data[:5]

[{'input_text': 'Correct this sentence: New and new technology has been introduced to the society .',
  'output_text': 'New technology has been introduced to society .'},
 {'input_text': 'Correct this sentence: One possible outcome is that an environmentally-induced reduction in motorization levels in the richer countries will outweigh any rise in motorization levels in the poorer countries .',
  'output_text': 'One possible outcome is that an environmentally-induced reduction in motorization levels in richer countries will outweigh any rise in motorization levels in poorer countries .'},
 {'input_text': 'Correct this sentence: Every person needs to know a bit about math , sciences , arts , literature and history in order to stand out in society .',
  'output_text': 'Every person needs to know a bit about math , science , art , literature and history in order to stand out in society .'},
 {'input_text': 'Correct this sentence: While the travel company will most likely show them some in

## combin

In [17]:

jfleg_dataset = Dataset.from_list(grammar_data)

tatoeba_small = tatoeba.select(range(5000))

combined_dataset = concatenate_datasets([tatoeba_small, jfleg_dataset])

print(combined_dataset[0])
print(len(combined_dataset))

{'translation': {'en': 'Pack your stuff.', 'fa': 'وسايلتو جمع کن باشه.'}, 'input_text': 'Translate English to Persian: Pack your stuff.', 'output_text': 'وسايلتو جمع کن باشه.'}
5747


In [18]:
from sklearn.model_selection import train_test_split


final_dataset = combined_dataset.train_test_split(test_size=0.1)
train_data = final_dataset["train"]
eval_data = final_dataset["test"]



In [19]:
final_dataset

DatasetDict({
    train: Dataset({
        features: ['translation', 'input_text', 'output_text'],
        num_rows: 5172
    })
    test: Dataset({
        features: ['translation', 'input_text', 'output_text'],
        num_rows: 575
    })
})

In [20]:
def preprocess(example):
    if "translation" in example and example["translation"] is not None:
        inp = example["translation"]["en"]
        out = example["translation"]["fa"]
    else:
        inp = example["input_text"]
        out = example["output_text"]
    return {"instruction": inp, "output": out}

processed_dataset = final_dataset.map(preprocess)


Map:   0%|          | 0/5172 [00:00<?, ? examples/s]

Map:   0%|          | 0/575 [00:00<?, ? examples/s]

In [21]:
processed_dataset

DatasetDict({
    train: Dataset({
        features: ['translation', 'input_text', 'output_text', 'instruction', 'output'],
        num_rows: 5172
    })
    test: Dataset({
        features: ['translation', 'input_text', 'output_text', 'instruction', 'output'],
        num_rows: 575
    })
})

In [22]:
type(processed_dataset['train']["translation"][0])

dict

In [23]:
print(processed_dataset["train"]["translation"][0])

{'en': 'Sundown, Greenie.', 'fa': 'غروب شده، تازه\u200cوارد.'}


In [24]:
def formatting_func(example):
    en_text = example["translation"][0]  # متن انگلیسی (اندیس 0)
    fa_text = example["translation"][1]  # متن فارسی (اندیس 1)
    formatted_text = f"Translate from English to Persian: {en_text} -> {fa_text}"
    return [formatted_text]  # بازگشت یک لیست با یک رشته

# Train

In [ ]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 122.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 115.4 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.4
    Uninstalling tokenizers-0.21.4:
      Successfully uninstalled tokenizers-0.21.4
  Attempting uninstall: transformers
    Found existing installation: transformers 4.55.4
    Uninstalling transformers-4.55.4:
      Successfully uninstalled transformers-4.55.4


In [31]:
from trl import SFTTrainer,SFTConfig
from transformers import TrainingArguments
import os

In [26]:
alpaca_prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""

In [27]:
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    outputs = examples["output"]
    texts = []
    for instruction, output in zip(instructions, outputs):
        # Format the prompt with instruction and output (for training, include output; for inference, only instruction)
        text = alpaca_prompt.format(instruction, output)
        texts.append(text)
    return {"text": texts}

processed_dataset = processed_dataset.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/5172 [00:00<?, ? examples/s]

Map:   0%|          | 0/575 [00:00<?, ? examples/s]

In [28]:
training_args = TrainingArguments(
    output_dir="./fine_tuned_model",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    warmup_steps=500,
    weight_decay=0.01,
    learning_rate=2e-4,
    fp16=True if torch.cuda.is_available() else False,
    logging_steps=100,
    eval_strategy="steps",  # تغییر از evaluation_strategy به eval_strategy
    eval_steps=500,
    save_steps=500,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    optim="adamw_torch",
    report_to="none",
    seed=42,
)

In [29]:
trainer = SFTTrainer(
    model=model,  # The PEFT (LoRA-adapted) model to fine-tune. Explanation: This is your Gemma model with LoRA layers added via Unsloth.
    tokenizer=tokenizer,  # The tokenizer for the model. Explanation: Handles text to token conversion; must match the model.
    train_dataset=processed_dataset["train"],  # The training dataset. Explanation: Your processed train split with ~5172 examples.
    eval_dataset=processed_dataset["test"],  # The evaluation dataset. Explanation: Your test split with ~575 examples for validation during training.
    dataset_text_field="text",  # The column in the dataset containing the formatted prompt text. Explanation: We added 'text' via formatting_prompts_func; SFTTrainer uses this for input.
    max_seq_length=max_seq_length,  # Maximum sequence length for inputs. Explanation: Set to 2048 as in your code; truncates/pads to this length.
    packing=True,  # Whether to pack multiple examples into one sequence for efficiency. Explanation: True helps with variable-length data, reducing padding and speeding up training (Unsloth supports this).
    args=training_args,  # The TrainingArguments object defined above. Explanation: Passes all the hyperparameters to the trainer.
)

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/5172 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/575 [00:00<?, ? examples/s]

In [32]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=processed_dataset["train"],
    eval_dataset=processed_dataset["test"],
    max_seq_length=2048,
    packing=False,
    formatting_func=formatting_func,  # <--- اینجا لیست بده
    args=SFTConfig(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=60,
        learning_rate=2e-4,
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="/content/drive/MyDrive/Mathematical_Foundations_and_Artificial_Intelligence_of_Source_Codes/fine-tune lora/Fine tune gemma/Model_w_save/gemma2b-grammar-Milad1304/gemma2b-grammar-Milad1304",
        report_to="none",
    ),
)


Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/5172 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/575 [00:00<?, ? examples/s]

In [33]:
trainer.train()


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 5,172 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 1,843,200 of 2,508,015,616 (0.07% trained)


Step,Training Loss,entropy
1,5.118000,0
2,4.769600,No Log
3,4.896100,No Log
4,4.456800,No Log
5,4.400500,No Log
6,4.221700,No Log
7,3.731500,No Log
8,4.180500,No Log
9,3.975100,No Log
10,3.604800,No Log


TrainOutput(global_step=60, training_loss=2.8992167890071867, metrics={'train_runtime': 102.1471, 'train_samples_per_second': 4.699, 'train_steps_per_second': 0.587, 'total_flos': 758704398925824.0, 'train_loss': 2.8992167890071867, 'epoch': 0.09280742459396751})

In [34]:
trainer.save_model("/content/drive/MyDrive/Mathematical_Foundations_and_Artificial_Intelligence_of_Source_Codes/fine-tune lora/Fine tune gemma/Model_w_save")
tokenizer.save_pretrained("/content/drive/MyDrive/Mathematical_Foundations_and_Artificial_Intelligence_of_Source_Codes/fine-tune lora/Fine tune gemma/Model_w_save")


('/content/drive/MyDrive/Mathematical_Foundations_and_Artificial_Intelligence_of_Source_Codes/fine-tune lora/Fine tune gemma/Model_w_save/tokenizer_config.json',
 '/content/drive/MyDrive/Mathematical_Foundations_and_Artificial_Intelligence_of_Source_Codes/fine-tune lora/Fine tune gemma/Model_w_save/special_tokens_map.json',
 '/content/drive/MyDrive/Mathematical_Foundations_and_Artificial_Intelligence_of_Source_Codes/fine-tune lora/Fine tune gemma/Model_w_save/tokenizer.model',
 '/content/drive/MyDrive/Mathematical_Foundations_and_Artificial_Intelligence_of_Source_Codes/fine-tune lora/Fine tune gemma/Model_w_save/added_tokens.json',
 '/content/drive/MyDrive/Mathematical_Foundations_and_Artificial_Intelligence_of_Source_Codes/fine-tune lora/Fine tune gemma/Model_w_save/tokenizer.json')

In [ ]:
trainer.train()